In [1]:
import numpy as np
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import pandas as pd
import json
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split

import urllib
import os
import csv
import cv2
import time
from PIL import Image
/
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

2023-04-08 16:39:37.222765: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-08 16:39:37.271777: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-08 16:39:37.272551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 16:39:38.572689: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#for test 
tf.__name__
print(tf.__version__)
print('1: ', tf.config.list_physical_devices('GPU'))
print('2: ', tf.test.is_built_with_cuda)
print('3: ', tf.test.gpu_device_name())
print('4: ', tf.config.get_visible_devices())
print('5: ',tf.test.is_built_with_rocm())
print('6: ',tf.test.is_built_with_cuda())


In [ ]:
# os.makedirs("snapshots", exist_ok=True)
# os.makedirs("models", exist_ok=True)

In [2]:
TRAINING_DATA_PATH = './'

with open(os.path.join(TRAINING_DATA_PATH, 'dataset.json')) as f:
   data = json.load(f)

dataset_df = pd.DataFrame(data)

In [3]:
dataset_df.head()

id                       created  width  \
0  2469da39-65f0-428b-a1de-cedbba7db32e  2018-07-11T07:27:39.1873304Z   1600   
1  22b40473-2892-4ab9-96be-9aee12ffd17a  2018-07-11T07:27:39.0942122Z   1600   
2  6209fb2c-ec39-4f8e-b86b-075cc181a81c  2018-07-11T07:27:38.2810459Z   1600   
3  13b9f109-f532-471a-8f39-f19eef551919  2018-07-11T07:27:38.2030543Z   1600   
4  f1f0f46b-4430-4227-ab3a-24ad1192728e  2018-07-11T07:27:38.1873131Z   1600   

   height imageUri thumbnailUri  \
0     564     None         None   
1     564     None         None   
2     270     None         None   
3     270     None         None   
4     277     None         None   

                                                tags  \
0  [{'tagId': '530d20f2-7260-42f1-bd42-bddca465c9...   
1  [{'tagId': '530d20f2-7260-42f1-bd42-bddca465c9...   
2  [{'tagId': '1599c0ca-83c0-4da7-b8c7-2a0a4b069f...   
3  [{'tagId': '1599c0ca-83c0-4da7-b8c7-2a0a4b069f...   
4  [{'tagId': '1599c0ca-83c0-4da7-b8c7-2a0a4b069f...   

                                             regions  
0  [{'regionId': '058d3c7a-a9fd-4c76-9f05-9eaff02...  
1  [{'regionId': '054c2b0c-a102-4908-9d09-fcf81b6...  
2  [{'regionId': '1609a4ed-81fa-4ef8-8dd8-7f45d84...  
3  [{'regionId': '115d74e0-6dd6-4b6d-b48b-9086018...  
4  [{'regionId': '0365e098-ca4a-432a-baf9-a4faa0b...

In [4]:
dataset_df['regions'].loc[0]

[{'regionId': '058d3c7a-a9fd-4c76-9f05-9eaff024c321',
  'tagName': 'CheckBox',
  'created': '2018-07-11T07:30:26.5948315Z',
  'tagId': 'd5cedd3f-acf2-4dce-8ead-e1a74e7e11d6',
  'left': 0.2646843,
  'top': 0.5903212,
  'width': 0.167400882,
  'height': 0.146875},
 {'regionId': '219ca3ac-6f74-45a8-913d-58127f057bd0',
  'tagName': 'Button',
  'created': '2018-07-11T07:30:26.5948315Z',
  'tagId': '69576625-a3c1-40bb-8947-936491484048',
  'left': 0.6182085,
  'top': 0.749696136,
  'width': 0.12334802,
  'height': 0.234375},
 {'regionId': '26714dc1-b8e4-4e7e-8ad7-ccf168a20755',
  'tagName': 'Label',
  'created': '2018-07-11T07:30:26.5948315Z',
  'tagId': 'b9944914-2e80-4b6a-8874-0fe55d073c45',
  'left': 0.0113803176,
  'top': 0.259071171,
  'width': 0.1938326,
  'height': 0.109375},
 {'regionId': '26ab6761-cb09-4d49-9688-f70231229a3f',
  'tagName': 'CheckBox',
  'created': '2018-07-11T07:30:26.5948315Z',
  'tagId': 'd5cedd3f-acf2-4dce-8ead-e1a74e7e11d6',
  'left': 0.6182085,
  'top': 0.39032

In [16]:
from collections import OrderedDict

dataset = OrderedDict()
dataset["image_name"] = list()
dataset["width"] = list()
dataset["height"] = list()
dataset["class_name"] = list()

dataset["x_min"] = list()
dataset["y_min"] = list()
dataset["x_max"] = list()
dataset["y_max"] = list()

for index, row in dataset_df.iterrows():
    image_path =  row.id + '.png'
    
   
    
    for region in row['regions']:
        x_min = int(region['left'] * row['width'])
        y_min = int(region['top'] * row['height'])

        width = region['width'] * row['width']
        height = region['height'] * row['height']
        x_max = int(x_min + width)
        y_max = int(y_min + height)
        
        dataset["width"].append(row.width)
        dataset["height"].append(row.height)
        dataset["image_name"].append(image_path)
        dataset['x_min'].append(x_min)
        dataset['y_min'].append(y_min)
        dataset['x_max'].append(x_max)
        dataset['y_max'].append(y_max)
        dataset['class_name'].append(region['tagName'])

In [17]:
df = pd.DataFrame(dataset)
df.head()

image_name  width  height   class_name  \
0  2469da39-65f0-428b-a1de-cedbba7db32e.png   1600     564     CheckBox   
1  2469da39-65f0-428b-a1de-cedbba7db32e.png   1600     564       Button   
2  2469da39-65f0-428b-a1de-cedbba7db32e.png   1600     564        Label   
3  2469da39-65f0-428b-a1de-cedbba7db32e.png   1600     564     CheckBox   
4  2469da39-65f0-428b-a1de-cedbba7db32e.png   1600     564  RadioButton   

   x_min  y_min  x_max  y_max  
0    423    332    690    414  
1    989    422   1186    554  
2     18    146    328    207  
3    989    220   1292    294  
4    804    107    952    168

In [27]:
# df['image_name'].loc[0]==df['image_name'].loc[100]
len(df)
df.to_csv('./images/annotations.csv',index=False)

In [ ]:
(df['class_name'].unique())

In [21]:
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = RANDOM_SEED)

In [22]:
train_df

image_name  width  height class_name  \
240   f66ced22-8f4e-4c35-8277-e49fba7bf192.png   1600     352   CheckBox   
1669  52840896-0073-4c1f-9aca-bbd87c17eac9.png   1200     900  Paragraph   
692   85c935d2-a69a-4d58-907b-22cf24a53fd5.png    900    1259      Label   
420   d20b2bf2-ee77-404f-8de3-9238284d1957.png   1111     900       Link   
1863  1803be9d-936c-4429-9301-2a0aaaffbb46.png   1200     900      Label   
...                                        ...    ...     ...        ...   
1130  8ded829b-7f25-4cd7-8d40-c1d3100d755d.png   1200     900   CheckBox   
1294  eaa0ceec-7f26-42b2-81a7-b8cd52416fec.png   1200     900      Image   
860   0f5761f9-a85c-4b7f-949a-189ab7cf853e.png    900    1200      Label   
1459  54f95c56-6e1e-42b9-b0ee-2f9de7134c6e.png   1200     900  Paragraph   
1126  8ded829b-7f25-4cd7-8d40-c1d3100d755d.png   1200     900      Label   

      x_min  y_min  x_max  y_max  
240      25    216    289    300  
1669    588    441    766    521  
692     181    989    633   1095  
420     740    253    862    346  
1863    364    203    493    257  
...     ...    ...    ...    ...  
1130    124    696    575    775  
1294    752    151    966    193  
860     208    753    390    813  
1459    162    769    383    869  
1126    614    139    840    208  

[1600 rows x 8 columns]

In [ ]:
ANNOTATIONS_FILE = "annotations.csv"
CLASSES_FILE = "classes.csv"

In [ ]:
train_df.to_csv(ANNOTATIONS_FILE, index = False, header = None)

In [ ]:
classes = set(['Label', 'Button', 'TextBox', 'ComboBox', 'Link', 'CheckBox', 'Heading', 'Image', 'Paragraph', 'RadioButton'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{}, {}\n'.format(line, i))

In [ ]:
# PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'
PRETRAINED_MODEL = './snapshots/resnet50_coco_best_v2.1.0.h5'

# URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
# urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [1]:
!head annotations.csv

./model/images/f66ced22-8f4e-4c35-8277-e49fba7bf192.png,25,216,289,300,CheckBox
./model/images/52840896-0073-4c1f-9aca-bbd87c17eac9.png,588,441,766,521,Paragraph
./model/images/85c935d2-a69a-4d58-907b-22cf24a53fd5.png,181,989,633,1095,Label
./model/images/d20b2bf2-ee77-404f-8de3-9238284d1957.png,740,253,862,346,Link
./model/images/1803be9d-936c-4429-9301-2a0aaaffbb46.png,364,203,493,257,Label
./model/images/bde0646d-e0d6-497f-b832-cb2bbf907eb1.png,390,347,644,437,TextBox
./model/images/3892666c-c7c7-4b69-9000-9f050d2ca7fb.png,440,170,637,231,ComboBox
./model/images/80ce0ae6-9f89-429b-b2da-ae1b25e45acb.png,67,411,210,474,Paragraph
./model/images/da54575a-4e89-49aa-af1a-c4d8490ba57b.png,597,359,670,448,Button
./model/images/770c6fbf-1046-4de3-a892-c1d53a9209ae.png,29,100,138,154,Label


In [ ]:
!python keras-retinanet/keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 500 --epochs 10 csv annotations.csv classes.csv

In [ ]:
!ls snapshots

In [ ]:
model_path = "snapshots/resnet50_csv_01.h5"

In [ ]:
# model = models.load_model(model_path, backbone_name="resnet50")
model = models.load_model(model_path)

model = models.convert_model(model)

In [ ]:
labels_to_name = pd.read_csv(CLASSES_FILE, header = None).T.loc[0].to_dict()
labels_to_name

In [ ]:
THRES_SCORE = 0.4

def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        continue

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_name[label], score)
    draw_caption(image, b, caption)

In [ ]:
THRES_SCORE = 0.4

def show_detected_objects(image_row):
    img_path = image_row.image_name
    true_box = [
        image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
    ]
    image = read_image_bgr(img_path)

    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
    boxes /= scale

    draw_box(draw, true_box, color=(255, 0, 0))
    draw_detections(image, boxes, scores, labels)
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < THRES_SCORE:
            continue;

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color = color)

        caption = "{} {:.3f}".format(labels_to_name[label], score)
        draw_caption(draw, b, caption)

    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [ ]:
test_df.head(n=10)

In [ ]:
for i in range(10, 20):
    show_detected_objects(df.iloc[i])

In [ ]:
dataset_df.loc[0]

In [ ]:
img_path='/mnt/sdb1/2.FILE/Computer Science/1.Extra/graduation project /1_datasets_gathered_v1/sketched-webpages-generator/_Output_Sketches v1/page_00000_2.jpg'

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

img=image.load_img(img_path)

x= image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)

In [ ]:
r = model.predict(img_data)
# print(r)

R= pd.DataFrame(r[0][0])

R
# r[0]

In [ ]:
R= pd.DataFrame(r[1])
R


In [ ]:
R= pd.DataFrame(r[2])
R
